In [ ]:
from math import ceil, exp
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = loadmat("carte_centreMetres.mat")
h_mat, x_val, y_val = data["h_MNT"], data["x_MNT"][0], data["y_MNT"][0]

In [ ]:
def h_mnt(x_float: float, y_float: float):
    x_floor, y_floor = (
        np.searchsorted(x_val, x_float) - 1,
        np.searchsorted(y_val, y_float) - 1,
    )
    alpha_x = (x_float - x_val[x_floor]) / (x_val[x_floor + 1] - x_val[x_floor])
    interp_y_floor = (
        alpha_x * h_mat[x_floor + 1, y_floor] + (1 - alpha_x) * h_mat[x_floor, y_floor]
    )
    interp_y_ceil = (
        alpha_x * h_mat[x_floor + 1, y_floor + 1]
        + (1 - alpha_x) * h_mat[x_floor, y_floor + 1]
    )
    alpha_y = (y_float - y_val[y_floor]) / (y_val[y_floor + 1] - y_val[y_floor])
    return alpha_y * interp_y_ceil + (1 - alpha_y) * interp_y_floor

In [ ]:
fig = plt.figure()
ax = fig.add_axes((0, 0, 1, 1))
ax.pcolormesh(x_val, y_val, h_mat)

In [ ]:
DT = 1
PHI = np.array(
    [
        [1, 0, 0, DT, 0, 0],
        [0, 1, 0, 0, DT, 0],
        [0, 0, 1, 0, 0, DT],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
    ]
)
CAP_AVION = 45 * np.pi / 180
V_AVION = 300 * 1000 / 3600
LONG_TRAJET = 60 * 1000
NB_IT = ceil(LONG_TRAJET / (DT * V_AVION))
VX = V_AVION * np.cos(CAP_AVION)
VY = V_AVION * np.sin(CAP_AVION)
ETAT_INIT = np.array([[150000], [150000], [8000], [VX], [VY], [0]])
SIGMA_BRUIT = 30
SIGMA_X = 3000
SIGMA_Y = 3000
SIGMA_Z = 500
SIGMA_VX = SIGMA_VY = SIGMA_VZ = 5
P_0 = np.diag(
    [
        SIGMA_X**2,
        SIGMA_Y**2,
        SIGMA_Z**2,
        SIGMA_VX**2,
        SIGMA_VY**2,
        SIGMA_VZ**2,
    ]
)
N_PART = 5000

In [ ]:
x_avion = ETAT_INIT[0] + VX * np.array(range(NB_IT))
y_avion = ETAT_INIT[1] + VY * np.array(range(NB_IT))
z_avion = ETAT_INIT[2] * np.ones(shape=(NB_IT,))
etat_avion = np.array([x_avion, y_avion, z_avion])

In [ ]:
fig = plt.figure()
ax = fig.add_axes((0, 0, 1, 1))
ax.set_ylim(y_val[-1], y_val[0])
ax.plot(etat_avion[0], etat_avion[1], color="red")
ax.pcolormesh(x_val, y_val, h_mat)

In [ ]:
rd = np.random.default_rng(20)
# gamma = np.linalg.cholesky(P_0)

In [ ]:
epsilon_bruit = rd.normal(loc=0, scale=SIGMA_BRUIT, size=NB_IT)
h_mes = (
    z_avion
    - np.array([h_mnt(x, y) for (x, y) in zip(x_avion, y_avion)])
    + epsilon_bruit
)

In [ ]:
epsilon: np.ndarray = rd.multivariate_normal(
    mean=6 * [0], cov=P_0, size=N_PART
).reshape((N_PART, 6, 1))

In [ ]:
particles = np.empty(shape=(NB_IT, N_PART, 6, 1))
weights = np.empty(shape=(NB_IT, N_PART))
particles[0] = ETAT_INIT + epsilon

In [ ]:
fig = plt.figure()
ax = fig.add_axes((0, 0, 1, 1))
ax.set_ylim(y_val[-1], y_val[0])
ax.plot(etat_avion[0], etat_avion[1], color="red")
ax.pcolormesh(x_val, y_val, h_mat)
ax.scatter(
    particles[0, :, 0, 0], particles[0, :, 1, 0], color="pink", marker=".", s=0.5
)

In [ ]:
for k in range(NB_IT - 1):
    part_pred = np.empty(shape=(N_PART, 6, 1))
    for i in range(N_PART):
        part_pred[i] = PHI @ particles[k, i, :, :]
        ecart = h_mes[k + 1] - (
            part_pred[i, 2, 0] - h_mnt(part_pred[i, 0, 0], part_pred[i, 1, 0])
        )
        weights[k + 1, i] = weights[k, i] * exp(
            -(1 / (2 * SIGMA_BRUIT**2)) * ecart**2
        )
    weights[k + 1, :] /= sum(weights[k + 1, :])